<a href="https://colab.research.google.com/github/nikshrimali/TSAI_END2_Phase1/blob/main/Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
!pip install faiss
!sudo apt-get install libomp-dev
!pip install transformers

     |████████████████████████████████| 4.7 MB 5.4 MB/s 
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libomp5
Suggested packages:
  libomp-doc
The following NEW packages will be installed:
  libomp-dev libomp5
0 upgraded, 2 newly installed, 0 to remove and 37 not upgraded.
Need to get 239 kB of archives.
After this operation, 804 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libomp5 amd64 5.0.1-1 [234 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libomp-dev amd64 5.0.1-1 [5,088 B]
Fetched 239 kB in 1s (300 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 2.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Re

In [ ]:
def seed_all(seed=42):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

In [ ]:
# BERT imports

from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm, trange
import pandas as pd
import io
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
% matplotlib inline

from transformers import BertTokenizer, BertModel
from transformers import BartConfig, BartTokenizer, BartForConditionalGeneration
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')

import faiss

import torch
from torch.optim import AdamW
from torch.utils.data import DataLoader
import glob


# specify GPU device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device == "cuda":
    n_gpu = torch.cuda.device_count()
    torch.cuda.get_device_name(0)
    device_name = tf.test.gpu_device_name()
    if device_name != '/device:GPU:0':
        raise SystemError('GPU device not found')
        print('Found GPU at: {}'.format(device_name))

else:
    print(f'Currently Running on {device}')

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
df_context = pd.read_csv('/content/drive/MyDrive/END2_CAPSTONE/context_groups.csv') 
df_merged = pd.read_csv('/content/drive/MyDrive/END2_CAPSTONE/merged_data.csv')
model_op = torch.load('/content/drive/MyDrive/END2_CAPSTONE/np_encoded_context.pt')

In [ ]:

def load_model(checkpoint, model, MODEL_STORE, q_string=''):

    checkpoints = sorted(glob.glob(f'{MODEL_STORE}/{checkpoint}*-[0-9]*'))
    global_step = int(checkpoints[0].split('-')[-1])
    ckpt_name = '{}-{}'.format(checkpoint, global_step)
    print("Loading model from checkpoint %s" % ckpt_name)
    
    PATH = f'{MODEL_STORE}/{ckpt_name}/{checkpoint}{q_string}-{str(global_step-1)}.pt'
    # PATH = f'{MODEL_STORE}/{ckpt_name}/q_checkpoint-{str(global_step-1)}.pt'
    model.load_state_dict(torch.load(PATH))
    print("model successfully loaded %s" % ckpt_name)

    return model

In [ ]:
class GetEncodings:
    def __init__(self, type='context'):
        MODEL_STORE = '/content/drive/MyDrive/END2_CAPSTONE'
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.model = BertModel.from_pretrained('bert-base-uncased')

        # Load model's pretrained weights
        if type == 'questions':
            self.model = load_model(checkpoint='qna_checkpoint', model=self.model, MODEL_STORE=MODEL_STORE, q_string='-q')
    

    def encode(self, text, max_length=200):
        '''Tokenize data and get encoded embeddings from model'''
        tk = self.tokenizer(
                text,
                max_length=max_length,
                padding='max_length',
                return_attention_mask=True,
                add_special_tokens=True,
                truncation=True,
                return_tensors="pt"
            )
        model_op = self.model(**tk)['pooler_output']
        return model_op

In [ ]:
class SearchSimilar:
    def __init__(self, iterator = None, filename=None, embeddings=None, shape=None, device="cpu"):

        self.iterator = iterator
        
        if os.path.exists(filename) == True:

            print(f'Index file {filename}')
            self.index = faiss.read_index(filename)  # index2 is identical to index
        
        else:

            self.index = faiss.index_factory(shape, "Flat", faiss.METRIC_INNER_PRODUCT)
            faiss.normalize_L2(embeddings)
            self.index.add(embeddings)
            faiss.write_index(self.index, filename)
            print(f'Index written at {filename}')
 
        
        if device == "cuda":
            print('Now running on CUDA')
            self.index = faiss.index_cpu_to_all_gpus(self.index)

  
        print(f'Index trained - {self.index.is_trained}')


    def get_n_similar_vectors(self, text, n=2):
        '''Find the top n similar vector from text using cosine similarity FAISS'''
        text = text.cpu().detach().numpy()
        faiss.normalize_L2(text)
        distance, index = self.index.search(text, n)

        context = []

        for i in index.tolist()[0]:
            context.append(self.iterator[i])
        return context
    
    def get_n_dissimilar_vectors(self, text, n=2):
        '''Find the top n dissimilar vector from text using cosine similarity FAISS'''
        pass

In [ ]:
class BARTTrain(torch.nn.Module):
    def __init__(self):
        super(BARTTrain, self).__init__()
        self.model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')
    
    def forward(self, encodings, mode):

        if mode == 'train':
            model_outputs = self.model(
                input_ids=encodings['input_ids_source_merged'].to(device), 
                attention_mask=encodings['attention_mask_source_merged'].to(device),
                decoder_input_ids=encodings['input_ids_target'].to(device),
                decoder_attention_mask=encodings['attention_mask_target'].to(device),
                labels=encodings['input_ids_target'].to(device))
        else:
            print('Running model in eval mode')
            model_outputs = self.model.generate(encodings)     

        return model_outputs

In [ ]:
# Inference
MODEL_STORE = '/content/drive/MyDrive/END2_CAPSTONE'

bart_model = BARTTrain().to(device)


def get_latest_checkpoint(checkpoint, model, MODEL_STORE, q_string=''):

    checkpoints = sorted(glob.glob(f'{MODEL_STORE}/{checkpoint}*-[0-9]*'))
    if len(checkpoints):
        global_step = int(checkpoints[0].split('-')[-1])
        ckpt_name = '{}-{}'.format(checkpoint, global_step)
        print("Loading model from checkpoint %s" % ckpt_name)
        
        PATH = f'{MODEL_STORE}/{ckpt_name}/{checkpoint}{q_string}-{str(global_step)}.pt'
        # PATH = f'{MODEL_STORE}/{ckpt_name}/q_checkpoint-{str(global_step-1)}.pt'
        model.load_state_dict(torch.load(PATH))
        print("model successfully loaded %s" % ckpt_name)
    
    else:
        print("No checkpoints available right now")
    return model

bart_model = get_latest_checkpoint('checkpoint', bart_model, MODEL_STORE)

Downloading:   0%|          | 0.00/558M [00:00<?, ?B/s]

Loading model from checkpoint checkpoint-103
model successfully loaded checkpoint-103


In [ ]:


def inference(question, bart_tokenizer, bart_model):

    # Get Pretrained BERT encodings

    ge = GetEncodings(type='questions')
    encoded_question = ge.encode(question, max_length=30)

    # Find top matching documents
    ss = SearchSimilar(iterator = df_context['context'].values.tolist(), filename='index.bin', embeddings=model_op, shape=768, device=device)
    similar_contexts = ss.get_n_similar_vectors(encoded_question, 3)
    similar_contexts.insert(0, question)

    combined_tokens = '</s></s>'.join(similar_contexts)

    print(f'Top similar document outputs is {combined_tokens}')

    # Prepare data for BART Inferencing

    source_encoding = tokenizer(
            combined_tokens,
            max_length=1024,
            padding='max_length',
            add_special_tokens=True,
            truncation=True,
            return_tensors="pt")
   

    # Inference BART Model
    output = bart_model(source_encoding['input_ids'].to(device), mode = 'eval')
    output = tokenizer.decode(output[0])
    print(output)
    return output


In [ ]:
tokens = inference('What does torch.cosine loss do in pytorch?', tokenizer, bart_model)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Loading model from checkpoint qna_checkpoint-1000
model successfully loaded qna_checkpoint-1000
Index file index.bin
Index trained - True
Top similar document outputs is What does torch.cosine loss do in pytorch?</s></s>You are manually setting weight, but what about bias?</s></s>Currently there is almost no support for `torch.complex32`, so yeah we should disable it.</s></s>Isn't `AverageUnpooling` very similar to `F.interpolate`?
Running model in eval mode
</s><s>bybyby by by by-by-by by- by- By-by</s>
